In [3]:
# Trick to import local code:
import sys
sys.path = ['/home/mp/michelj/src/sensorsio/src'] + sys.path

In [4]:
import rasterio as rio
import numpy as np
from sensorsio import sentinel2, utils
from importlib import reload
from matplotlib import pyplot

reload(sentinel2)

<module 'sensorsio.sentinel2' from '/home/mp/michelj/src/sensorsio/src/sensorsio/sentinel2.py'>

# Creating a dataset

In [5]:
dataset = sentinel2.Sentinel2('/datalake/S2-L2A-THEIA/31TDH/2019/05/31/SENTINEL2B_20190531-105916-927_L2A_T31TDH_C_V2-2/')

In [6]:
dataset

SENTINEL2B, 2019-05-31 00:00:00, T31TDH

In [14]:
dataset.date, dataset.year, dataset.day_of_year

(datetime.datetime(2019, 5, 31, 0, 0), 2019, 151)

In [9]:
dataset.tile

'T31TDH'

In [10]:
dataset.crs

CRS.from_epsg(32631)

In [11]:
dataset.bounds

BoundingBox(left=399960.0, bottom=4690200.0, right=509760.0, top=4800000.0)

In [12]:
dataset.satellite

<Satellite.S2B: 'SENTINEL2B'>

# Reading bands

In [4]:
arr = dataset.read_bands(sentinel2.Sentinel2.GROUP_10M)

In [5]:
arr.shape

(4, 10980, 10980)

In [1]:
arr_rgb, _, _ = utils.rgb_render(arr, bands=[2,1,0], 
                                 dmin=np.array([0.,0., 0.]), 
                                 dmax=np.array([0.2,0.2,0.2]))
pyplot.imshow(arr_rgb)

NameError: name 'utils' is not defined

In [ ]:
psfs = dataset.PSF([sentinel2.Sentinel2.B2,sentinel2.Sentinel2.B3])

In [ ]:
pyplot.imshow(psfs[0,...])

In [ ]:
dataset.build_mask_path(sentinel2.Sentinel2.CLM)

In [ ]:
dataset = sentinel2.Sentinel2('/datalake/S2-L2A-THEIA/31TDH/')

In [ ]:
import os
pn = os.path.basename(os.path.normpath('/datalake/S2-L2A-THEIA/31TDH/2019/05/31/SENTINEL2B_20190531-105916-927_L2A_T31TDH_C_V2-2/'))

In [ ]:
import dateutil
d=dateutil.parser.parse(pn[11:19],fuzzy=True)

In [ ]:
d.year

In [ ]:
d.timetuple().tm_yday

In [ ]:
d='/datalake/S2-L2A-THEIA/31TDH/2019/05/31/SENTINEL2B_20190531-105916-927_L2A_T31TDH_C_V2-2'

In [ ]:
import glob
glob.glob(f"{d}/*FRE_{band.value}.tif")